# Tuning FLORIS using ModelFit

Demonstrate tuning of FLORIS to the SCADA data using the ModelFit object

In [1]:
from pathlib import Path

import pandas as pd

from flasc.data_processing.dataframe_manipulations import (
    is_day_or_night,
    plot_sun_altitude_with_day_night_color,
)

import matplotlib.pyplot as plt
import numpy as np

from floris import UncertainFlorisModel

import flasc.model_fitting.floris_tuning as ft
from flasc.analysis import energy_ratio as er
from flasc.analysis.analysis_input import AnalysisInput
from flasc.utilities.tuner_utilities import resim_floris
from flasc.utilities.utilities_examples import load_floris_smarteole

import logging
import pickle
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import optuna
from flasc.data_processing.dataframe_manipulations import is_day_or_night
from flasc.model_fit.cost_library import turbine_power_error_abs
from flasc.model_fit.model_fit import ModelFit
from flasc.model_fit.opt_library import atomic_opt_optuna, opt_optuna_with_unc
from floris import ParFlorisModel, UncertainFlorisModel

# You can use Matplotlib instead of Plotly for visualization by simply replacing `optuna.visualization` with
# `optuna.visualization.matplotlib` in the following examples.
from optuna.visualization.matplotlib import (
    plot_contour,
    plot_edf,
    plot_intermediate_values,
    plot_optimization_history,
    plot_parallel_coordinate,
    plot_param_importances,
    plot_rank,
    plot_slice,
    plot_timeline,
)


## Read data

In [2]:
root_path = Path.cwd()
f = root_path / "postprocessed" / "df_scada_data_60s_filtered_and_northing_calibrated.pkl"
df_scada = pd.read_pickle(f)

In [3]:
latitude = 49.8435
longitude = 2.801556

# Compute day/night in default settings and plot
df_scada = is_day_or_night(df_scada, latitude, longitude)

In [4]:
# Limit SCADA data to region of wake steering

# Specify offsets
start_of_offset = 200  # deg
end_of_offset = 240  # deg

# Limit SCADA to this region
df_scada = df_scada[
    (df_scada.wd_smarteole > (start_of_offset - 20))
    & (df_scada.wd_smarteole < (end_of_offset + 20))
]

In [5]:
# Assign wd, ws and pow ref and subset SCADA based on reference variables used
# in the SMARTEOLE wake steering experiment (TODO reference the experiment)
df_scada = df_scada.assign(
    wd=lambda df_: df_["wd_smarteole"],
    ws=lambda df_: df_["ws_smarteole"],
    pow_ref=lambda df_: df_["pow_ref_smarteole"],
)

In [6]:
# For tuning grab the reference, control and test turbines
ref_turbs = [0, 1, 2, 6]
test_turbs = [4]
control_turbs = [5]

## Split the data


In [7]:
# Split SCADA into baseline and wake steeering (controlled)
df_scada_baseline = df_scada[df_scada.control_mode == "baseline"]
df_scada_controlled = df_scada[df_scada.control_mode == "controlled"]

In [8]:
df_scada_baseline_day = df_scada_baseline[df_scada_baseline.is_day]
df_scada_baseline_night = df_scada_baseline[~df_scada_baseline.is_day]
df_scada_controlled_day = df_scada_controlled[df_scada_controlled.is_day]
df_scada_controlled_night = df_scada_controlled[~df_scada_controlled.is_day]

## Load FLORIS Model

In [9]:
fm, _ = load_floris_smarteole(wake_model="emgauss")
D = fm.core.farm.rotor_diameters[0]

#### Assume uncertain model

In [10]:
fm = UncertainFlorisModel(fm, wd_std=3.0, wd_resolution=2.0, ws_resolution=0.25)

# Baseline tuning

In [11]:
# Just tune the first wake expansion parameter
parameter_list = [
            (
                "wake",
                "wake_velocity_parameters",
                "empirical_gauss",
                "wake_expansion_rates",
            )
        ]

parameter_name_list = [
    "we_1",
]

parameter_range_list = [
    (0.0, 0.2),
]

parameter_index_list = [0]

In [12]:
mf = ModelFit(
        df_scada_baseline,
        fm,
        turbine_power_error_abs,
        parameter_list=parameter_list,
        parameter_name_list=parameter_name_list,
        parameter_range_list=parameter_range_list,
        parameter_index_list=parameter_index_list,
    )

In [ ]:

# Compute the default cost
print("Evaluating cost with default parameters")
default_cost = mf.evaluate_floris()
print(f"Default cost: {default_cost}")

Evaluating baseline cost
Baseline cost: 15744146.432581067


In [14]:
# Optimization
n_trials = 20
opt_result, study = atomic_opt_optuna(mf, timeout=None, n_trials=n_trials)

[I 2025-04-07 21:14:46,514] A new study created in memory with name: ModelFit
[I 2025-04-07 21:14:49,359] Trial 0 finished with value: 15744146.432581067 and parameters: {'we_1': 0.01}. Best is trial 0 with value: 15744146.432581067.
[I 2025-04-07 21:14:52,144] Trial 1 finished with value: 17018999.581567045 and parameters: {'we_1': 0.10976270078546496}. Best is trial 0 with value: 15744146.432581067.
[I 2025-04-07 21:14:54,911] Trial 2 finished with value: 17336127.660370752 and parameters: {'we_1': 0.1430378732744839}. Best is trial 0 with value: 15744146.432581067.
[I 2025-04-07 21:14:57,658] Trial 3 finished with value: 17135252.203355674 and parameters: {'we_1': 0.12055267521432877}. Best is trial 0 with value: 15744146.432581067.
[I 2025-04-07 21:15:00,490] Trial 4 finished with value: 17010106.527407728 and parameters: {'we_1': 0.10897663659937938}. Best is trial 0 with value: 15744146.432581067.
[I 2025-04-07 21:15:03,395] Trial 5 finished with value: 16689668.770873036 and par

## Tune the deflection model

In [17]:
# Just tune the first wake expansion parameter
parameter_list = [
            (
        "wake",
        "wake_deflection_parameters",
        "empirical_gauss",
        "horizontal_deflection_gain_D",
            )
        ]

parameter_name_list = [
    "deflection_gain",
]

parameter_range_list = [
    (0.0, 5.0),
]

parameter_index_list = [None]

In [18]:
# Set the yaw angle matrix
yaw_vec = df_scada_controlled.wind_vane_005

yaw_angles = np.zeros((yaw_vec.shape[0], 7))
yaw_angles[:, control_turbs[0]] = yaw_vec

In [20]:
mf = ModelFit(
        df_scada_controlled,
        fm,
        turbine_power_error_abs,
        parameter_list=parameter_list,
        parameter_name_list=parameter_name_list,
        parameter_range_list=parameter_range_list,
        parameter_index_list=parameter_index_list,
        yaw_angles=yaw_angles,
    )

In [ ]:
# Compute the default cost
print("Evaluating cost with default parameters")
default_cost = mf.evaluate_floris()
print(f"Default cost: {default_cost}")

Evaluating cost with default parameters
Default cost: 15496862.862766583


In [22]:
# Optimization
n_trials = 20
opt_result, study = atomic_opt_optuna(mf, timeout=None, n_trials=n_trials)

[I 2025-04-07 21:16:47,160] A new study created in memory with name: ModelFit
[I 2025-04-07 21:16:53,779] Trial 0 finished with value: 15496862.862766583 and parameters: {'deflection_gain': 3.0}. Best is trial 0 with value: 15496862.862766583.
[I 2025-04-07 21:17:00,307] Trial 1 finished with value: 15492154.108956836 and parameters: {'deflection_gain': 2.7440675196366238}. Best is trial 1 with value: 15492154.108956836.
[I 2025-04-07 21:17:07,024] Trial 2 finished with value: 15510094.082506895 and parameters: {'deflection_gain': 3.5759468318620975}. Best is trial 1 with value: 15492154.108956836.
[I 2025-04-07 21:17:13,599] Trial 3 finished with value: 15497131.480597619 and parameters: {'deflection_gain': 3.0138168803582195}. Best is trial 1 with value: 15492154.108956836.
[I 2025-04-07 21:17:20,105] Trial 4 finished with value: 15491803.25647524 and parameters: {'deflection_gain': 2.724415914984484}. Best is trial 4 with value: 15491803.25647524.
[I 2025-04-07 21:17:26,609] Trial 5